In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run CCDC over a tiled grid

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/3-CCDC_and_Scaling_Over_Large_Areas">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3-CCDC_and_Scaling_Over_Large_Areas">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3-CCDC_and_Scaling_Over_Large_Areas">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This notebook uses Landsat in the Continuous Change Detection and Classification (CCDC) temporal segmentation algorithm over a tiled grid in order to minimize memory errors

Learn more about [CCDC](https://www.sciencedirect.com/science/article/abs/pii/S0034425714000248).

### Objective

In this tutorial, you learn how to create and manipulate CCDC outputs.

This tutorial uses the following Google Cloud services:

- `Google Earth Engine`

The steps performed include:

- Create tile mesh grid over a study area
- Creating CCDC outputs over the mesh grid
- Manipulating EE array image objects to get meaningful data out of CCDC outputs

In [24]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import inspect
# from IPython.display import IFrame,display, HTML
ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Done


## Before you begin

### Set your current URL under `workbench_url`
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/`

### Set a folder to use for all exports under `export_path_root` 
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [4]:
workbench_url = 'https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/'
export_path_root  = 'projects/rcr-gee/assets/lcms-training'

print('Done')

Done


In [6]:
# Create folder and a collection and make public

export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'
aml.create_asset(export_ccdc_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_ccdc_collection,writers = [],all_users_can_read = True,readers = [])

# aml.batchCopy('projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4',export_ccdc_collection,outType = 'imageCollection')
# aml.batchDelete('projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4')
# print('Done')

Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC
['projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID91-52_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID91-53_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID91-54_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID92-52_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID92-53_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4/CCDC_Tile-60000m_ID93-52_yrs1984-2023_jds1-365', 'projects/rcr-gee/assets/tiledOutputs-lcms-traini

In [8]:
Map.clearMap()
Map.proxy_url = workbench_url
# First, we'll view the tiles used in the current CONUS LCMS workflow
lcms_CONUS_composites = ee.ImageCollection('projects/lcms-tcc-shared/assets/Composites/Composite-Collection-yesL7-1984-2020')\
                                                .filter(ee.Filter.calendarRange(2022,2022,'year'))

# Pull the geometry of each tile in the composites
lcms_composites_tile_geo = lcms_CONUS_composites.map(lambda f:ee.Feature(f.geometry()).copyProperties(f,['studyAreaName']))

# Add the tiles and a composite for reference
Map.addLayer(lcms_CONUS_composites.mosaic(),getImagesLib.vizParamsTrue10k,'Example CONUS 2022 LCMS Composite')
Map.addLayer(lcms_composites_tile_geo,{},'LCMS Composite Tile Geometry')

Map.centerObject(lcms_composites_tile_geo)
Map.turnOnInspector()
Map.view()

Adding layer: Example CONUS 2022 LCMS Composite
Adding layer: LCMS Composite Tile Geometry
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [9]:
Map.clearMap()

# Here is an example of how to create a pyramid of tiles at various scales
# Generally you would start with the biggest tile possible and work your way down till you stop having memory issues
# Currently, LCMS uses 480km tiles (with a 900m buffer) for most processing
lcms_CONUS_studyArea = ee.FeatureCollection('projects/lcms-292214/assets/CONUS-Ancillary-Data/conus')
lcms_CONUS_projection = lcms_CONUS_composites.first().projection()

def getGrid(studyArea,projection,size):
  grid = studyArea.geometry().coveringGrid(projection.atScale(size))
  Map.addLayer(grid,{},'Tile Grid {}m'.format(size))
  return grid

grid480= getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,480000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,240000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,120000)
getGrid(lcms_CONUS_studyArea,lcms_CONUS_projection,60000)
Map.addLayer(lcms_CONUS_studyArea,{},'LCMS CONUS Study Area')

Map.turnOnInspector()
Map.view()


Adding layer: Tile Grid 480000m
Adding layer: Tile Grid 240000m
Adding layer: Tile Grid 120000m
Adding layer: Tile Grid 60000m
Adding layer: LCMS CONUS Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [14]:
Map.clearMap()
# As an example of how to use this approach, we first need to get each available tile into a list
# Then we would iterate across each tile and get the data and export, etc...
ids = grid480.limit(2).aggregate_histogram('system:index').keys().getInfo()
for id in ids:
  # Get the tile and clip it to the study area and then buffer
  tile = grid480.filter(ee.Filter.eq('system:index',id)).geometry().intersection(lcms_CONUS_studyArea,240,lcms_CONUS_projection).buffer(900)
  Map.addLayer(tile,{},'Tile {}'.format(id))

Map.view()

Adding layer: Tile -3,-6
Adding layer: Tile -4,-6
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [16]:
# Since CCDC is the most memory intensive algorith we use, you are the most likely to need to use this approach when running CCDC
# The previous CCDC example ran from 2010-2023 and did not run out of memory
# If you run it from 1984 to 2023 using just Landsat, you are much more likely to run out of memory

Map.clearMap()

# First let's set up some Landsat and CCDC parameters

#Update the startJulian and endJulian variables to indicate your seasonal
#constraints. This supports wrapping for tropics and southern hemisphere.
#If using wrapping and the majority of the days occur in the second year, the system:time_start will default
#to June 1 of that year.Otherwise, all system:time_starts will default to June 1 of the given year
#startJulian: Starting Julian date
#endJulian: Ending Julian date
startJulian = 1
endJulian = 365

#Specify start and end years for all analyses
#More than a 3 year span should be provided for time series methods to work
#well. If providing pre-computed stats for cloudScore and TDOM, this does not
#matter
startYear = 1984
endYear = 2023


#Choose whether to include Landat 7
#Generally only included when data are limited
includeSLCOffL7 = True


#Which bands/indices to export
#These will not always be used to find breaks - that is specified below in the ccdcParams
#Options are: ["blue","green","red","nir","swir1","swir2","NDVI","NBR","NDMI","NDSI","brightness","greenness","wetness","fourth","fifth","sixth","tcAngleBG"]
#Be sure that any bands in ccdcParams.breakpointBands are in this list
exportBands = ["blue","green","red","nir","swir1","swir2","NDVI"]

###############################################################
#CCDC Params
# ccdcParams ={
#   breakpointBands:['green','red','nir','swir1','swir2','NDVI'],//The name or index of the bands to use for change detection. If unspecified, all bands are used.//Can include: 'blue','green','red','nir','swir1','swir2'
#                                                               //'NBR','NDVI','wetness','greenness','brightness','tcAngleBG'
#   tmaskBands : None,//['green','swir2'],//The name or index of the bands to use for iterative TMask cloud detection. These are typically the green band and the SWIR2 band. If unspecified, TMask is not used. If specified, 'tmaskBands' must be included in 'breakpointBands'.,
#   minObservations: 6,//Factors of minimum number of years to apply new fitting.
#   chiSquareProbability: 0.99,//The chi-square probability threshold for change detection in the range of [0, 1],
#   minNumOfYearsScaler: 1.33,//Factors of minimum number of years to apply new fitting.,\
#   lambda: 0.002,//Lambda for LASSO regression fitting. If set to 0, regular OLS is used instead of LASSO
#   maxIterations : 25000, //Maximum number of runs for LASSO regression convergence. If set to 0, regular OLS is used instead of LASSO.
#   dateFormat : 1 //'fractional' (1) is the easiest to work with. It is the time representation to use during fitting: 0 = jDays, 1 = fractional years, 2 = unix time in milliseconds. The start, end and break times for each temporal segment will be encoded this way.

# };
ccdcParams ={
  'breakpointBands':['green','red','nir','swir1','swir2','NDVI'],
  'tmaskBands' : None,
  'minObservations': 6,
  'chiSquareProbability': 0.99,
  'minNumOfYearsScaler': 1.33,
  'lambda': 0.002,
  'maxIterations' : 25000,
  'dateFormat' : 1
};



# We'll use the full Puerto Rico and US Virgin Islands LCMS study area
studyArea = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/Ancillary/prusvi_boundary_buff2mile')

# Set the size (in meters) of the tiles
tileSize = 60000

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None
projection = ee.Projection(crs,transform)

# Get the grid
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))


#Remove any extremely high band/index values
def removeGT1(img):
  lte1 = img.select(['blue','green','nir','swir1','swir2']).lte(1).reduce(ee.Reducer.min());
  return img.updateMask(lte1);

getImagesLib.vizParamsFalse['min']=0.15
getImagesLib.vizParamsFalse['max']=0.8


Map.addLayer(studyArea,{},'Study Area')
Map.turnOnInspector()
Map.centerObject(studyArea)
Map.view()

Adding layer: Tile Grid 60000m
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [17]:
# Now, we'll iterate across each tile and run CCDC


# As an example of how to use this approach to create composites
ids = grid.aggregate_histogram('system:index').keys().getInfo()
for id in ids:
    print(id)
    # Get the tile and buffer it so there are no missing pixels at tile edges
    tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)
    
    
    # Map.addLayer(tile,{},'Tile {}'.format(id))
    
    processedScenes = getImagesLib.getProcessedLandsatScenes(studyArea = tile,startYear = startYear, endYear = endYear,
                                                        startJulian = startJulian,endJulian = endJulian,
                                                        includeSLCOffL7 = includeSLCOffL7).select(exportBands)
    processedScenes = processedScenes.map(removeGT1)
    # print(processedScenes.size().getInfo())
    
    #Set the scene collection in the ccdcParams
    ccdcParams['collection'] = processedScenes

    #Run CCDC
    ccdc = ee.Image(ee.Algorithms.TemporalSegmentation.Ccdc(**ccdcParams))
    ccdc = ccdc.set({'startYear':startYear,
                     'endYear':endYear,
                     'startJulian':startJulian,
                     'endJulian':endJulian,
                     'TileSize':tileSize,
                     'TileID':id})
    
    
    exportName = 'CCDC_Tile-{}m_ID{}_yrs{}-{}_jds{}-{}'.format(tileSize,id.replace(',','-'),startYear,endYear,startJulian,endJulian)
    exportPath = f'{export_ccdc_collection}/{exportName}'
    print(exportPath)

    getImagesLib.exportToAssetWrapper(ccdc,exportName,exportPath,{'.default':'sample'},tile,scale,crs,transform,overwrite=False)



90,54
Get Processed Landsat: 
Start date: Jan 01 1984 , End date: Dec 31 2023
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
projects/rcr-gee/assets/lcms-training/lcms-training_module-3_CCDC/CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365
pyramiding object: {'.default': 'sample'}
CCDC_Tile-60000m_ID90-54_yrs1984-2023_jds1-365 currently exists or is being exported and overwrite = False. Export not started. Set overwite = True if you would like to overwite any existing asset or asset exporting task
91,52
Get Processed Landsat: 
Start date: Jan 01 1984 , End date: Dec 31 2023
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors fo

In [67]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(exportPathRoot, type = 'imageCollection')

print('done')

done


In [25]:
Map.clearMap()

# Bring in the outputs and mosaic them into a single image
ccdcImg = ee.ImageCollection(export_ccdc_collection).mosaic()
Map.addLayer(ccdcImg,{},'CCDC Raw Image')

Map.turnOnInspector()
Map.view()

Adding layer: CCDC Raw Image
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [26]:
# We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
# While CCDC is not as good at change detection as LandTrendr, the breaks can be used for detecting changes in phenology
print(inspect.getsource(changeDetectionLib.ccdcChangeDetection))

def ccdcChangeDetection(ccdcImg,bandName):
  magKeys = ['.*_magnitude']
  tBreakKeys = ['tBreak']
  changeProbKeys = ['changeProb']
  changeProbThresh = 1

  #Pull out pieces from CCDC output
  magnitudes = ccdcImg.select(magKeys)
  breaks = ccdcImg.select(tBreakKeys)
  
  #Map.addLayer(breaks.arrayLength(0),{'min':1,'max':10});
  changeProbs = ccdcImg.select(changeProbKeys)
  changeMask = changeProbs.gte(changeProbThresh)
  magnitudes = magnitudes.select(bandName + '.*')

  
  #Sort by magnitude and years
  breaksSortedByMag = breaks.arraySort(magnitudes)
  magnitudesSortedByMag = magnitudes.arraySort()
  changeMaskSortedByMag = changeMask.arraySort(magnitudes)
  
  breaksSortedByYear = breaks.arraySort()
  magnitudesSortedByYear = magnitudes.arraySort(breaks)
  changeMaskSortedByYear = changeMask.arraySort(breaks)
  
  #Get the loss and gain years and magnitudes for each sorting method
  highestMagLossYear = breaksSortedByMag.arraySlice(0,0,1).arrayFlatten([['loss_year']])
  highestM

In [27]:



#Specify which band to use for loss and gain.
#This is most important for the loss and gain magnitude since the year of change will be the same for all years
changeDetectionBandName = 'NDVI'

# Choose whether to show the most recent ('mostRecent') or highest magnitude ('highestMag') CCDC break
sortingMethod = 'mostRecent'
####################################################################################################
Map.clearMap()
#We will not look at more useful ways of visualizing CCDC outputs
#First, we will extract the change years and magnitude
changeObj = changeDetectionLib.ccdcChangeDetection(ccdcImg,changeDetectionBandName);
Map.addLayer(changeObj[sortingMethod]['loss']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.addLayer(changeObj[sortingMethod]['loss']['mag'],{'min':-0.5,'max':-0.1,'palette':changeDetectionLib.lossMagPalette},'Loss Mag',False);
Map.addLayer(changeObj[sortingMethod]['gain']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.gainYearPalette},'Gain Year');
Map.addLayer(changeObj[sortingMethod]['gain']['mag'],{'min':0.05,'max':0.2,'palette':changeDetectionLib.gainMagPalette},'Gain Mag',False);

Map.turnOnInspector()
Map.view()
#Double click on map to see raw years of loss and gain breaks
#Notice as you zoom in the layers change since GEE is processing outputs at a given pyramid level

Adding layer: Loss Year
Adding layer: Loss Mag
Adding layer: Gain Year
Adding layer: Gain Mag
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


In [29]:
# We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
# This function is useful to get the harmonic model coefficient values for a given date image 
print(inspect.getsource(changeDetectionLib.getCCDCSegCoeffs) )

def getCCDCSegCoeffs(timeImg,ccdcImg,fillGaps):
  coeffKeys = ['.*_coefs']
  tStartKeys = ['tStart']
  tEndKeys = ['tEnd']
  tBreakKeys = ['tBreak']
  
  #Get coeffs and find how many bands have coeffs
  coeffs = ccdcImg.select(coeffKeys)
  bns = coeffs.bandNames()
  nBns = bns.length()
  harmonicTag = ee.List(['INTP','SLP','COS1','SIN1','COS2','SIN2','COS3','SIN3'])

   
  #Get coeffs, start and end times
  coeffs = coeffs.toArray(2)
  tStarts = ccdcImg.select(tStartKeys)
  tEnds = ccdcImg.select(tEndKeys)
  tBreaks = ccdcImg.select(tBreakKeys)
  
  #If filling to the tBreak, use this
  tStarts = ee.Image(ee.Algorithms.If(fillGaps,tStarts.arraySlice(0,0,1).arrayCat(tBreaks.arraySlice(0,0,-1),0),tStarts))
  tEnds = ee.Image(ee.Algorithms.If(fillGaps,tBreaks.arraySlice(0,0,-1).arrayCat(tEnds.arraySlice(0,-1,None),0),tEnds))
  
  
  #Set up a mask for segments that the time band intersects
  tMask = tStarts.lt(timeImg).And(tEnds.gte(timeImg)).arrayRepeat(1,1).arrayRepeat(2,1)
  coeffs = 

In [28]:
# We will need to manipulate the array image to get meaningful data such as synthetic composites and harmonic coefficients
# This function is useful to get the predicted values for a given date
print(inspect.getsource(changeDetectionLib.simpleCCDCPrediction) )

def simpleCCDCPrediction(img,timeBandName,whichHarmonics,whichBands):
  #Unit of each harmonic (1 cycle)
  omega = ee.Number(2.0).multiply(math.pi)

  #Pull out the time band in the yyyy.ff format
  tBand = img.select([timeBandName])
  
  #Pull out the intercepts and slopes
  intercepts = img.select(['.*_INTP'])
  slopes = img.select(['.*_SLP']).multiply(tBand)
  
  #Set up the omega for each harmonic for the given time band
  tOmega = ee.Image(whichHarmonics).multiply(omega).multiply(tBand)
  cosHarm = tOmega.cos()
  sinHarm = tOmega.sin()
  
  #Set up which harmonics to select

  harmSelect = ee.List(whichHarmonics).map(lambda n: ee.String('.*').cat(ee.Number(n).format()))
  
  #Select the harmonics specified
  sins = img.select(['.*_SIN.*'])
  sins = sins.select(harmSelect)
  coss = img.select(['.*_COS.*'])
  coss = coss.select(harmSelect)
  
  #Set up final output band names
  outBns = ee.List(whichBands).map(lambda bn: ee.String(bn).cat('_predicted'))
  
  #Iterate across each ban

In [30]:
# Same workflow follows as with an untiled CCDC output
Map.clearMap()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False

#Apply the CCDC harmonic model across a time series
#First get a time series of time images with a time step of 0.1 of a year
yearImages = changeDetectionLib.getTimeImageCollection(startYear,endYear,startJulian,endJulian,0.1);

#Then predict the CCDC models
fitted = changeDetectionLib.predictCCDC(ccdcImg,yearImages,fillGaps,whichHarmonics)
Map.addLayer(fitted.select(['.*_predicted']),{'bands':'swir1_predicted,nir_predicted,red_predicted','min':0.05,'max':0.6},'Fitted CCDC',True);


# Synthetic composites visualizing
# Take common false color composite bands and visualize them for the next to the last year

# First get the bands of predicted bands and then split off the name
fittedBns = fitted.select(['.*_predicted']).first().bandNames()
bns = fittedBns.map(lambda bn: ee.String(bn).split('_').get(0))

# Filter down to the next to the last year and a summer date range
syntheticComposites = fitted.select(fittedBns,bns)\
    .filter(ee.Filter.calendarRange(endYear-1,endYear-1,'year'))\
    .filter(ee.Filter.calendarRange(60,80)).first()

# Visualize output as you would a composite
Map.addLayer(syntheticComposites,getImagesLib.vizParamsFalse,'Synthetic Composite')

Map.turnOnInspector()
Map.view()


Adding layer: Fitted CCDC
Adding layer: Synthetic Composite
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://23dcc4ff89e513fb-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None
